In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skspatial.objects import LineSegment, Line, Vector

from enum import Enum
Layer = Enum("Layer", "FRONT BACK")

from helpers import *
from pcb_json import *

# Parameters 
edit those:

In [ ]:
# Track width and spacing
TRACK_WIDTH = 0.127
TRACK_SPACING = 0.127

# via defaults
VIA_DIAM = 0.8
VIA_DRILL = 0.4

# this is for a 1.27mm pitch pin
PIN_DIAM = 1.0
PIN_DRILL = 0.65

SCREW_HOLE_DRILL_DIAM = 2.3  # 2.3mm drill for a 2mm screw

# this is for the PCB connector - see https://www.farnell.com/datasheets/2003059.pdf
PAD_WIDTH = 3
PAD_HEIGHT = 2
PAD_PITCH = 2.5

# how to connect coils
PAD_ENABLE = False
CONNECT_WITH_VIAS = True

# NET Naming
COIL_NET_NAME = "coil" 
USE_INDIVIDUAL_NET_NAMES_PER_COIL = False # appends numbering to COIL_NET_NAME
USE_ABC_NET_NAMES_FOR_COILS = True # appends A,B,C to COIL_NET_NAME

# draw on edge cuts:
PCB_EDGE_CUTS = False

LAYERS = 4

# Geometry RADIUS_
# -------------------------------------------------
RADIUS_STATOR_HOLE          = 20 # 14
RADIUS_CONNECTIONS_INSIDE   = RADIUS_STATOR_HOLE + 3 * TRACK_SPACING # for connecting the bottoms of the coils

RADIUS_COIL_START           = 33
RADIUS_COIL_CENTER          = 38
RADIUS_COIL_CENTER_VIA = RADIUS_COIL_CENTER + 0.5
RADIUS_COIL_END             = 40

RADIUS_CONNECTIONS_OUTSIDE  = 53
RADIUS_CONNECTOR            = 53

RADIUS_TOTAL_STATOR         = 55

SCREW_HOLE_RADIUS = RADIUS_TOTAL_STATOR # where to put the mounting pins

# many coils (are placed ccw)
NUM_SEGMENTS = 12
NUM_COILS = 12

ROTATION = 0

space = RADIUS_COIL_START * np.sin(np.deg2rad(360 / NUM_COILS / 2)) 
TURNS = int(space / (TRACK_SPACING+TRACK_WIDTH))

FILE_NAME = f"coil_motor_{RADIUS_TOTAL_STATOR}mm.json"

# meta helper
coil_windings_width = TURNS * (TRACK_WIDTH+TRACK_SPACING)
radius_coil_start_effective = round(RADIUS_COIL_START - coil_windings_width, 2)
radius_coil_end_effective = round(RADIUS_COIL_END + coil_windings_width,2)

print(TURNS)
print(f"Effective radius range: {radius_coil_start_effective} - {radius_coil_end_effective}")

# Arbitrary Coil Generation

In [ ]:
# templates must be convex, simetric around the X axis and must include the center points on both size (e.g. (X1, 0).... (X2, 0) )
template = [
    (-3.5, 0),
    (-3.5, -0.01),
    (1.9, -1.45),
    (1.9, 0.0),
    (1.9, 1.45),
    (-3.5, 0.01),
]

def plot_points(template):
    df = pd.DataFrame(template, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="blue")

    scatter_df = pd.DataFrame(template, columns=["x", "y"])
    scatter_df.plot.scatter(x="x", y="y", color="red", ax=ax)

    ax.axis("equal")
    plt.grid(True)
    
    ax.text(0.05, 0.95, f"len= {len(template)}", transform=ax.transAxes, ha="left")

plot_points(template)

In [ ]:
def calculate_point(point, p_here, p_next, spacing, turn):
    vector = Vector(p_here) - Vector(p_next)
    normal = vector / np.linalg.norm(vector)
    normal = np.array([-normal[1], normal[0]])  # rotate 90 degrees

    reference_vector = Vector([-100, 0])
    angle = np.rad2deg(Vector(point).angle_between(reference_vector))
    if point[1] > 0:
        angle = 360 - angle

    # move the  point along the normal vector by the spacing
    offset = spacing * (turn * 360 + angle) / 360
    coil_point = point + normal * offset
    return (coil_point[0], coil_point[1])


def get_coil(template, turns, spacing):
    coil_points = []
    reference_vector = Vector([-100, 0])

    for turn in range(turns):
        for index in range(len(template)):
            p_here = template[index]
            turn_here = turn

            p_next = template[(index + 1) % len(template)]

            turn_here = turn
            turn_next = (turn * len(template) + index + 1) // len(template)

            coil_p_here = calculate_point(p_here, p_here, p_next, spacing, turn_here)
            coil_p_next = calculate_point(p_next, p_here, p_next, spacing, turn_next)

            if len(coil_points) >= 2:
                
                line1 = Line(
                    coil_points[-2],
                    np.array(coil_points[-1]) - np.array(coil_points[-2]),
                )
                # create a line from the two new points
                line2 = Line(
                    np.array(coil_p_here),
                    np.array(np.array(coil_p_here) - np.array(coil_p_next)),
                )

                # find the intersection of the two lines
                try:  # replace the previous point with the intersection
                    intersection = line1.intersect_line(line2)
                    coil_points[-1] = intersection
                except:  # the lines did not intersect so just add the points
                    coil_points.append(coil_p_here)

                coil_points.append(coil_p_next)
            else:
                coil_points.append(coil_p_here)
                coil_points.append(coil_p_next)

    return coil_points

def coil(template): 
    return get_coil(template, TURNS, TRACK_SPACING + TRACK_WIDTH)


# Generate a single coil

In [ ]:
x_from = -3.5
x_to = 1.9
y_amp = 1.45

radius_len = RADIUS_COIL_END - RADIUS_COIL_START

angle = round(360/2/NUM_COILS,2)
y_amp = round(np.sin(np.deg2rad(angle)) * radius_len, 2)
x_len = round(np.cos(np.deg2rad(angle)) * radius_len, 2)

x_to = RADIUS_COIL_END - RADIUS_COIL_CENTER
x_from = x_to - x_len

micro = 0.01 #1e-6
template = [
    (x_from, 0),
    (x_from, -micro),
    (x_to, -y_amp),
    (x_to, 0.0),
    (x_to, y_amp),
    (x_from, +micro)
]
shape = [
    (x_from+x_len/2,    -y_amp/2),
    (x_to,              -y_amp*0.8),
    (x_to,              0),
    (x_from+x_len/2,    +y_amp/2),
    (x_to,              +y_amp*0.8),
]
shape = [
    (x_to, -y_amp),
    (x_to, 0.0),
    (x_to, y_amp),
]
# template = shape
# template.insert(0, (x_from, 0))
# template.insert(1, (x_from, -micro))
# # # for i in range(len(shape)):
# # #     j = len(shape)-i-1
# # #     template.append([shape[j][0], -shape[j][1]])
# template.append((x_from, +micro))
print(template)

if 1:
    print(template)

    plot_points(template)
    plot_points(optimize_points(coil(template)))
    plot_points(chaikin_(coil(template), 5))

    ###############
    template_f = []
    for i in range(len(template)):
        template_f.append(template[len(template) - i - len(template) // 2])
        # print(f"i = {i}, map = {len(template) - i - len(template) // 2}, template_f = {template_f[i]}")

    template_f = flip_x(template_f)

    points_top = chaikin_(flip_x(coil(template_f)), 5)
    points_bot = chaikin_(coil(template), 5)

    # add point to middle for via
    points_top = [(RADIUS_COIL_CENTER_VIA - RADIUS_COIL_CENTER, 0)] + points_top
    points_bot = [(RADIUS_COIL_CENTER_VIA - RADIUS_COIL_CENTER, 0)] + points_bot

    def plot_tracks_vias(track_points_top, track_points_bot, via_points=0):

        df = pd.DataFrame(track_points_top, columns=["x", "y"])
        ax = df.plot.line(x="x", y="y", color="red")
        df = pd.DataFrame(track_points_bot, columns=["x", "y"])
        ax = df.plot.line(x="x", y="y", color="blue", ax=ax)

        if via_points:
            scatter_df = pd.DataFrame(via_points, columns=["x", "y"])
            scatter_df.plot.scatter(x="x", y="y", color="green", ax=ax)

        ax.axis("equal")
        plt.grid(True)
        ax.text(0.05, 0.95, f"len= {len(template)}", transform=ax.transAxes, ha="left")


    plot_tracks_vias(points_top, points_bot, 0)

# Reproduce coils

In [ ]:
vias = []
tracks_top = []
tracks_bot = []
pads = []
pins = []
mounting_holes = []
silk = []
components = []

arc_seg = 360 / NUM_COILS

USE_LABELS = False #todo

def angleAt(coil_index):
    return coil_index * arc_seg + ROTATION

def place_coil(coil_points, angle, radius):
    return translate(rotate(coil_points, angle), radius, angle)

# def appendTo(tracks_obj, name, pts):
#     tracks_obj.append({"net": name, "pts": pts})
    
# the main coils
coil_labels = ["A", "B", "C"]
coils_top = []
coils_bot = []
for i in range(NUM_SEGMENTS):
    angle = angleAt(i)
    radius = RADIUS_COIL_CENTER
    if (i // 3) % 2 == 0:
        coil_top = place_coil(points_top, angle, radius)
        coil_bot = place_coil(points_bot, angle, radius)
    else:
        # slightly nudge the coils so that they don't overlap when flipped
        coil_top = place_coil(flip_y(points_top), angle, radius)
        coil_bot = place_coil(flip_y(points_bot), angle, radius)
        
    coils_top.append(coil_top)
    coils_bot.append(coil_bot)

    name = COIL_NET_NAME + "_"
    if USE_INDIVIDUAL_NET_NAMES_PER_COIL:
        name += str(i).zfill(2)
    if USE_ABC_NET_NAMES_FOR_COILS:
        name += coil_labels[i % 3]

    tracks_top.append({"net": name, "pts": coil_top})
    tracks_bot.append({"net": name, "pts": coil_bot})
    
    vias.append(create_via(pol2cat(angle, RADIUS_COIL_CENTER_VIA), name))
    silk.append(create_silk(pol2cat(angle, RADIUS_COIL_CENTER), coil_labels[i % 3]))
    # silk.append(create_silk(pol2cat(angle, RADIUS_TOTAL_STATOR), coil_labels[i % 3]))
    silk.append(create_silk(pol2cat(angle+1.5, radius_coil_end_effective+2), coil_labels[i % 3]))

if 1:
    via_points = []
    for via in vias:
        via_points.append([via["x"], via["y"]])

    track_points_top = []
    for t in tracks_top:
        for p in t["pts"]:
            track_points_top.append(p)

    track_points_bot = []
    for t in tracks_bot:
        for p in t["pts"]:
            track_points_bot.append([p[0], p[1]])
            
    plot_tracks_vias(track_points_top, track_points_bot, via_points)
        




# Create coil inner connections

In [ ]:
# all ___At functions are little helpers that take coil_index + sometimes radius
def pointAt(coil_index, radius):
    return pol2cat(angleAt(coil_index), radius)

def nameAt(coil_index):
    name = COIL_NET_NAME
    if USE_ABC_NET_NAMES_FOR_COILS:
        name += coil_labels[coil_index % 3]
    return name

def appendAt(coils_ref, coil_index, radius):
    coils_ref[coil_index].append(pointAt(coil_index, radius))

def appendVia(coil_index, radius):
    vias.append(create_via(pointAt(coil_index, radius), nameAt(coil_index)))

radius_coni_common = RADIUS_CONNECTIONS_INSIDE

# connects coils with arc and uses last points of coils to connect to the arc
def connect_coils_auto_arc(c1, c2 , tracks_ref, radius):
    tracks_ref.append({
        "net": nameAt(c1),
        "pts": (
            [coils_bot[c1][-1]]
            + draw_arc(angleAt(c1), angleAt(c2), radius)
            + [coils_bot[c2][-1]] )})     
    
def connect_coils_with_arc(c1, c2 , tracks_ref, radius, radius_points, 
                           append_coils = True, 
                           append_vias = False):
    pts = draw_arc(angleAt(c1), angleAt(c2), radius)

    if radius_points != 0:
        pts = [pointAt(c1, radius_points)] + pts + [pointAt(c2, radius_points)]

    tracks_ref.append({
        "net": nameAt(c1),
        "pts": pts}) 
    
    if append_coils != 0:
        appendAt(coils_top, c1, radius)
        appendAt(coils_top, c2, radius)

    if append_vias:
        appendVia(c1, radius)
        appendVia(c2, radius)

if 1: # create coil ic = inner connections

    space = 5 * TRACK_SPACING 

    radius_coni_A = radius_coni_common + 3 * space + VIA_DIAM / 2
    for c in range(0, 12, 3):
        point = pointAt(c, radius_coni_A)
        coils_bot[c].append(point)
        vias.append(create_via(point, nameAt(c)))
    for con in range(2):
        connect_coils_auto_arc(con*6+0, con*6+3, tracks_top, radius_coni_A)
     
    radius_coni_B = radius_coni_common
    for c in range(1, 12, 3):
        point = pointAt(c, radius_coni_B)
        coils_bot[c].append(point)
        # vias.append(create_via(point, nameAt(c)))
    for con in range(2):
        connect_coils_auto_arc(con*6+1, con*6+4, tracks_bot, radius_coni_B)
        
    radius_coni_C = radius_coni_common + 2 * space - VIA_DIAM / 2
    for c in range(2, 12, 3):
        point = pointAt(c, radius_coni_C)
        coils_bot[c].append(point)
        vias.append(create_via(point, nameAt(c)))
    for con in range(2):
        connect_coils_auto_arc(con*6+2, con*6+5, tracks_top, radius_coni_C)

### OUTSIDE CONNECTIONS
# ------------------------------------------------------------------------------------
if 1:
    # connects the middle/star point
    r = RADIUS_CONNECTIONS_OUTSIDE
    r_inc = - 5 * TRACK_SPACING 
    connect_coils_with_arc(9, 11, tracks_top, r, 0, 
                           append_coils=True, 
                           append_vias=False)
    appendAt(coils_top, 10, r)

    r += r_inc
    connect_coils_with_arc(3, 6, tracks_top, r, 0,
                           append_coils=True, 
                           append_vias=False)

    r += r_inc
    connect_coils_with_arc(4, 7, tracks_bot, r, 0,
                           append_coils=True, 
                           append_vias=True)
    r += r_inc
    connect_coils_with_arc(5, 8, tracks_bot, r, 0,
                           append_coils=True, 
                           append_vias=True)

#  create the pads for connecting the inputs to the coils
if PAD_ENABLE:

    def appendSilk(y, text, size=1, angle=0):
        silk.append(create_silk((RADIUS_CONNECTOR - PAD_HEIGHT - 2.5, y), text, "f", 2.5, -900))

    appendSilk(+PAD_PITCH,   "C")
    appendSilk( 0,           "B")
    appendSilk(-PAD_PITCH,   "A")
    
    def appendPad(y, name):
        pads.append(create_pad([RADIUS_CONNECTOR, y], PAD_WIDTH, PAD_HEIGHT, "a", name))
    
    appendPad(+PAD_PITCH,   nameAt(0))
    appendPad(0,            nameAt(1))
    appendPad(-PAD_PITCH,   nameAt(2))

    # connect coil A to the top pad
    pad_connection_point_x = RADIUS_CONNECTOR
    pad_angle = np.rad2deg(np.arcsin(PAD_PITCH / pad_connection_point_x))
    coils_top[0].append(pointAt(0, pad_connection_point_x))
    appendVia(0, pad_connection_point_x)

    # connect coil B to the middle pad
    coils_top[1].append((pad_connection_point_x + PAD_WIDTH / 2 + VIA_DIAM / 2, 0))
    vias.append(create_via(
            ((pad_connection_point_x + PAD_WIDTH / 2 + VIA_DIAM / 2, 0)), nameAt(1)))
    
    # connect coil C to the bottom pad
    coils_top[2].append(pointAt(2, pad_connection_point_x))
    appendVia(2, pad_connection_point_x)

elif CONNECT_WITH_VIAS:
    for i in range(3):
        appendAt(coils_top, i, RADIUS_CONNECTOR)
        appendVia(i, RADIUS_CONNECTOR)

# Multi-Layer

In [ ]:
# if we are doing multiple layers then duplicate the front and back layers
tracks_in = []
if LAYERS >= 4:
    tracks_in.append(tracks_bot.copy())
    tracks_in.append(tracks_top.copy())
if LAYERS >= 6:
    tracks_in.append(tracks_bot.copy())
    tracks_in.append(tracks_top.copy())
if LAYERS == 8:
    tracks_in.append(tracks_bot.copy())
    tracks_in.append(tracks_top.copy())

# Generate JSON

In [ ]:
# Generate the JSON output file
if PAD_ENABLE:
    # these final bits of wiring up to the input pads don't need to be duplicated
    tracks_bot.append({
            "net": COIL_NET_NAME,
            "pts": [
                (pad_connection_point_x + PAD_WIDTH / 2, 0),
                (pad_connection_point_x, 0),
            ],
        })
    tracks_bot.append({
            "net": COIL_NET_NAME,
            "pts": draw_arc(angleAt(0), -pad_angle, pad_connection_point_x, 1),
        })
    tracks_bot.append({
            "net": COIL_NET_NAME,
            "pts": draw_arc(angleAt(2), pad_angle, pad_connection_point_x, 1),
        })

nibble_angle_size = 360 * SCREW_HOLE_DRILL_DIAM / (2 * np.pi * RADIUS_TOTAL_STATOR)

if PCB_EDGE_CUTS:
    outer_cuts = (
        draw_arc(
            -45 + nibble_angle_size / 2, 45 - nibble_angle_size / 2, RADIUS_TOTAL_STATOR, 5
        )
        + translate(
            rotate(draw_arc(5, 175, SCREW_HOLE_DRILL_DIAM / 2, 5)[::-1], 135),
            RADIUS_TOTAL_STATOR,
            45,
        )
        + draw_arc(
            45 + nibble_angle_size / 2, 135 - nibble_angle_size / 2, RADIUS_TOTAL_STATOR, 5
        )
        + translate(
            rotate(draw_arc(5, 175, SCREW_HOLE_DRILL_DIAM / 2, 5), 225)[::-1],
            RADIUS_TOTAL_STATOR,
            135,
        )
        + draw_arc(
            135 + nibble_angle_size / 2, 225 - nibble_angle_size / 2, RADIUS_TOTAL_STATOR, 5
        )
        + translate(
            rotate(draw_arc(5, 175, SCREW_HOLE_DRILL_DIAM / 2, 5), 315)[::-1],
            RADIUS_TOTAL_STATOR,
            225,
        )
        + draw_arc(
            225 + nibble_angle_size / 2, 315 - nibble_angle_size / 2, RADIUS_TOTAL_STATOR, 5
        )
        + translate(
            rotate(draw_arc(5, 175, SCREW_HOLE_DRILL_DIAM / 2, 5), 45)[::-1],
            RADIUS_TOTAL_STATOR,
            315,
        )
    )

    edge_cuts = [
        outer_cuts,
        draw_arc(0, 360, RADIUS_STATOR_HOLE, 1),
    ]
else:
    edge_cuts = []

# dump out the json version
json_result = dump_json(
    filename=FILE_NAME,
    track_width=TRACK_WIDTH,
    pin_diam=PIN_DIAM,
    pin_drill=PIN_DRILL,
    via_diam=VIA_DIAM,
    via_drill=VIA_DRILL,
    vias=vias,
    pins=pins,
    pads=pads,
    silk=silk,
    tracks_f=tracks_top,
    tracks_in=tracks_in,
    tracks_b=tracks_bot,
    mounting_holes=mounting_holes,
    edge_cuts=edge_cuts,
    components=components,
)

In [ ]:
plot_json(json_result, 70)
print(json_result['parameters'])

print(TURNS)
print(f"Effective radius range: {radius_coil_start_effective} - {radius_coil_end_effective}")